## Create option chain dataframes from source

In [ ]:
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0] / "src")


if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
import os
import re
import warnings

import numpy as np
import pandas as pd
from MMAR.MMAR import MMAR  # noqa: E402
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")

In [2]:
# Get all the entries in raw
dir_list = os.listdir("../data/raw/")

In [3]:
# Select just the needed ones
dir_list = sorted([x for x in dir_list if x.startswith("spy_eod_")])

In [4]:
dir_list

['spy_eod_2018-7fpgd6',
 'spy_eod_2019-eotnx0',
 'spy_eod_2020-kwe0mi',
 'spy_eod_2021-borwkq',
 'spy_eod_2022q1-ww0cra',
 'spy_eod_2022q2-xofvwa',
 'spy_eod_2022q3-we1um1',
 'spy_eod_2022q4-wh2csq',
 'spy_eod_2023q1-zfoivd',
 'spy_eod_2023q2-nnkjka',
 'spy_eod_2023q3-nyvdmv',
 'spy_eod_2023q4-vzwoxb']

In [5]:
def clean_df(
    df: pd.DataFrame,
    suffix: str | None = None,
    dest_path: str = "../data/processed/chain",
) -> pd.DataFrame:
    # Fix column names
    new_cols = []
    for col in df.columns:
        new_col = col.strip().replace("[", "").replace("]", "")
        new_cols.append(new_col)
    df.columns = new_cols

    # Drop unused columns
    df = df.drop(
        columns=[
            "QUOTE_UNIXTIME",
            "QUOTE_READTIME",
            "QUOTE_TIME_HOURS",
            "EXPIRE_UNIX",
        ]
    )

    # Fix column types
    num_cols = [
        "C_DELTA",
        "C_GAMMA",
        "C_VEGA",
        "C_THETA",
        "C_RHO",
        "C_IV",
        "C_VOLUME",
        "C_LAST",
        "C_BID",
        "C_ASK",
        "P_BID",
        "P_ASK",
        "P_LAST",
        "P_DELTA",
        "P_GAMMA",
        "P_VEGA",
        "P_THETA",
        "P_RHO",
        "P_IV",
        "P_VOLUME",
    ]

    for col in num_cols:
        df[col] = df[col].replace(" ", np.nan)
        df[col] = df[col].astype(float)

    # Fix date coulmns
    df.QUOTE_DATE = pd.to_datetime(df.QUOTE_DATE)
    df.EXPIRE_DATE = pd.to_datetime(df.EXPIRE_DATE)

    # Sort df
    df = df.sort_values(by=["QUOTE_DATE", "EXPIRE_DATE"]).reindex()

    # Save df
    df.to_csv(dest_path + suffix + ".csv.gz", index=False, compression="gzip")

    return df

In [6]:
def create_chain(path: str, orig_path: str = "../data/raw/") -> pd.DataFrame:
    dir_path = orig_path + path + "/"
    file_list = os.listdir(dir_path)
    df = None
    for file in tqdm(file_list):
        print(f"Rading {dir_path+file}")
        tmp_df = pd.read_csv(dir_path + file, low_memory=False)
        if df is None:
            df = tmp_df
        else:
            df = pd.concat([df, tmp_df])

    return df

In [7]:
for directory in tqdm(dir_list):
    suffix = re.search(r"(?<=spy_eod_)(.*?)(?=-)", directory).group()
    _ = clean_df(create_chain(directory), suffix)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201808.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201809.txt


Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201802.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201803.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201801.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201810.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201804.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201805.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201811.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201807.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201812.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201806.txt


  0%|          | 0/12 [00:00<?, ?it/s]

...

Rading ../data/raw/spy_eod_2023q2-nnkjka/spy_eod_202304.txt
Rading ../data/raw/spy_eod_2023q2-nnkjka/spy_eod_202305.txt
Rading ../data/raw/spy_eod_2023q2-nnkjka/spy_eod_202306.txt


  0%|          | 0/3 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2023q3-nyvdmv/spy_eod_202308.txt
Rading ../data/raw/spy_eod_2023q3-nyvdmv/spy_eod_202309.txt
Rading ../data/raw/spy_eod_2023q3-nyvdmv/spy_eod_202307.txt


  0%|          | 0/3 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2023q4-vzwoxb/spy_eod_202310.txt
Rading ../data/raw/spy_eod_2023q4-vzwoxb/spy_eod_202311.txt
Rading ../data/raw/spy_eod_2023q4-vzwoxb/spy_eod_202312.txt


### Create option price file

We create a file to be used for ML models

In [5]:
# Get all the entries in processed
dir_list = os.listdir("../data/processed/")

In [6]:
# Select just the needed ones
dir_list = sorted([x for x in dir_list if x.startswith("chain20")])

In [7]:
dir_list

['chain2018.csv.gz',
 'chain2019.csv.gz',
 'chain2020.csv.gz',
 'chain2021.csv.gz',
 'chain2022q1.csv.gz',
 'chain2022q2.csv.gz',
 'chain2022q3.csv.gz',
 'chain2022q4.csv.gz',
 'chain2023q1.csv.gz',
 'chain2023q2.csv.gz',
 'chain2023q3.csv.gz',
 'chain2023q4.csv.gz']

In [8]:
def create_opt_cahin_price(filepath) -> pd.DataFrame:
    opt_chain = pd.read_csv(filepath, parse_dates=True)
    # Get option prices
    opt_prices = opt_chain[["QUOTE_DATE", "UNDERLYING_LAST"]].drop_duplicates()
    opt_prices["QUOTE_DATE"] = pd.to_datetime(opt_prices["QUOTE_DATE"])
    opt_prices.columns = ["Date", "Last"]
    opt_prices = opt_prices.set_index("Date")

    return opt_prices

In [9]:
final_df = None
for filename in tqdm(dir_list):
    filepath = "../data/processed/" + filename
    print(f"Reading {filepath}")
    tmp_df = create_opt_cahin_price(filepath)
    if final_df is None:
        final_df = tmp_df
    else:
        final_df = pd.concat([final_df, tmp_df])

  0%|          | 0/12 [00:00<?, ?it/s]

Reading ../data/processed/chain2018.csv.gz
Reading ../data/processed/chain2019.csv.gz
Reading ../data/processed/chain2020.csv.gz
Reading ../data/processed/chain2021.csv.gz
Reading ../data/processed/chain2022q1.csv.gz
Reading ../data/processed/chain2022q2.csv.gz
Reading ../data/processed/chain2022q3.csv.gz
Reading ../data/processed/chain2022q4.csv.gz
Reading ../data/processed/chain2023q1.csv.gz
Reading ../data/processed/chain2023q2.csv.gz
Reading ../data/processed/chain2023q3.csv.gz
Reading ../data/processed/chain2023q4.csv.gz


In [10]:
final_df = final_df.sort_index()

final_df.to_csv("../data/processed/last_quote.csv")

## Create file for ML models

In [11]:
def generate_diff(num: int, max_iter: int = 6) -> int:  # type: ignore
    """Generator to create alternate sequence

    Example:
    generate_diff(21) returns 21,20,22,19,23,18,...

    Args:
        num (int): starting point
        max_iter (int): maximum number of iterations

    Returns:
        int: next element

    Yields:
        Iterator[int]: number
    """
    start = num
    i = 1
    while True:
        yield num
        if i % 2 == 0:
            num = start + i // 2
        else:
            num = start - (i // 2 + 1)
        i += 1
        if i > max_iter:
            break

In [12]:
def get_ml_df(
    df: pd.DataFrame, dates_df: pd.DataFrame, diff_date: int = 21
) -> pd.DataFrame:
    # Returning df
    result = df.head(0).copy()

    # Loop for date
    for row in tqdm(dates_df.itertuples(), total=len(dates_df)):
        quote_date, last_quote = row
        quote_date = quote_date.strftime("%Y-%m-%d")
        # Get chain for the date
        query_df = df.query("(QUOTE_DATE==@quote_date)")
        # display(query_df)
        for date_shift in generate_diff(diff_date, 8):  # noqa: B007
            wk_query = query_df.query("(diff_date==@date_shift)")

            if len(wk_query) > 0:
                # Get Strike
                # Index of the minimum difference
                idx = wk_query["diff"].argmin()
                # Append result
                result = pd.concat((result, wk_query.iloc[idx : idx + 1]))
                break
    return result

In [13]:
# Get all the entries in processed
dir_list = os.listdir("../data/processed/")

In [14]:
# Select just the needed ones
dir_list = sorted([x for x in dir_list if x.startswith("chain20")])

In [15]:
dir_list

['chain2018.csv.gz',
 'chain2019.csv.gz',
 'chain2020.csv.gz',
 'chain2021.csv.gz',
 'chain2022q1.csv.gz',
 'chain2022q2.csv.gz',
 'chain2022q3.csv.gz',
 'chain2022q4.csv.gz',
 'chain2023q1.csv.gz',
 'chain2023q2.csv.gz',
 'chain2023q3.csv.gz',
 'chain2023q4.csv.gz']

In [16]:
ml_df = None
for filename in tqdm(dir_list):
    filepath = "../data/processed/" + filename
    print(f"Reading {filepath}")
    tmp_df = pd.read_csv(filepath, parse_dates=True)
    tmp_df["diff"] = np.abs(tmp_df["UNDERLYING_LAST"] - tmp_df["STRIKE"])
    tmp_df["diff_date"] = (
        pd.to_datetime(tmp_df["EXPIRE_DATE"]) - pd.to_datetime(tmp_df["QUOTE_DATE"])
    ).dt.days
    tmp_df = tmp_df[
        [
            "QUOTE_DATE",
            "EXPIRE_DATE",
            "UNDERLYING_LAST",
            "STRIKE",
            "C_LAST",
            "C_BID",
            "C_ASK",
            "C_IV",
            "C_VOLUME",
            "diff",
            "diff_date",
        ]
    ]
    ret_df = get_ml_df(tmp_df, final_df)

    if ml_df is None:
        ml_df = ret_df
    else:
        ml_df = pd.concat([ml_df, ret_df])

  0%|          | 0/12 [00:00<?, ?it/s]

Reading ../data/processed/chain2018.csv.gz


  0%|          | 0/1508 [00:00<?, ?it/s]

Reading ../data/processed/chain2019.csv.gz


  0%|          | 0/1508 [00:00<?, ?it/s]

...

  0%|          | 0/1508 [00:00<?, ?it/s]

Reading ../data/processed/chain2023q3.csv.gz


  0%|          | 0/1508 [00:00<?, ?it/s]

Reading ../data/processed/chain2023q4.csv.gz


  0%|          | 0/1508 [00:00<?, ?it/s]

In [17]:
ml_df

,QUOTE_DATE,EXPIRE_DATE,UNDERLYING_LAST,STRIKE,C_LAST,C_BID,C_ASK,C_IV,C_VOLUME,diff,diff_date
423,2018-01-02,2018-01-24,268.81,269.0,1.56,1.72,1.76,0.07134,80.0,0.19,22
2680,2018-01-03,2018-01-24,270.47,270.0,2.15,2.10,2.15,0.07361,271.0,0.47,21
4917,2018-01-04,2018-01-24,271.56,272.0,1.67,1.57,1.61,0.06871,392.0,0.44,20
7390,2018-01-05,2018-01-26,273.41,273.0,2.21,2.23,2.28,0.07675,NaN,0.41,21
9795,2018-01-08,2018-01-31,273.94,274.0,2.08,2.07,2.09,0.07593,490.0,0.06,23
...,...,...,...,...,...,...,...,...,...,...,...
230651,2023-12-22,2024-01-12,473.68,474.0,5.70,5.75,5.78,0.11611,1917.0,0.32,21
234602,2023-12-26,2024-01-19,475.61,476.0,6.12,6.15,6.17,0.10991,3007.0,0.39,24
238524,2023-12-27,2024-01-19,476.47,476.0,6.19,6.15,6.23,0.10420,2636.0,0.47,23
242357,2023-12-28,2024-01-19,476.73,477.0,5.46,5.52,5.54,0.10172,2850.0,0.27,22


In [18]:
# Set date index
ml_df = ml_df.set_index(pd.to_datetime(ml_df["QUOTE_DATE"]))

In [19]:
# Rename index
ml_df.index.name = "Date"

In [20]:
# Sort df
ml_df = ml_df.sort_index()

In [21]:
# Save to file
ml_df.to_csv("../data/processed/ml_df.csv")

## Generate expected price using Monte Carlo simulation

In [22]:
import scipy.stats as st

In [23]:
# Get prices
spy = pd.read_csv("../data/raw/spy.csv", parse_dates=True, index_col=0)

In [24]:
# Get the working dataframe
ml_df = pd.read_csv("../data/processed/ml_df.csv", parse_dates=True, index_col=0)

Get interest rate (form Fred St Louis: https://fred.stlouisfed.org/series/DGS1MO)

In [25]:
interest_rate = pd.read_csv(
    "../data/raw/DGS1MO.csv", parse_dates=True, index_col=0, na_values=["."]
)

In [26]:
# Fill NaNs
interest_rate = interest_rate.ffill()

In [27]:
assert interest_rate.isna().sum().sum() == 0

In [28]:
# Shift one: we want to rely on yesterday's rate
interest_rate = interest_rate.shift().dropna()

In [29]:
def get_call_price(prices, strike, r, T):
    values = prices[prices > strike]
    prob = len(values) / len(prices)
    exp_val = np.mean(values)
    # print(f"Probability expected price {exp_val:.2f} > {strike:.2f} is {prob:.2%}")
    return np.exp(-r * T) * prob * (exp_val - strike), prob

In [30]:
exp_price = []  # the expected underlying value after n steps
exp_call_prices = []  # the expecte call price
date_idx = []  # date index
rates = []  # Interest rates
strikes = []  # strike price
last_quotes = []  # previous underlying value
days = []  # Number of days to expiration
probs = []  # probability that exp_price > strike at expiration
# Some other statistical properties
exp_price_min = []
exp_price_max = []
exp_price_std = []
exp_price_median = []
exp_price_q1 = []
exp_price_q3 = []
exp_price_kurtosis = []
exp_price_skew = []
num_sim = 500_000
prev_ir = 0
for row in tqdm(ml_df.itertuples(), total=len(ml_df)):
    # Get elements
    final_date = row.Index
    S0 = row.UNDERLYING_LAST
    num_steps = row.diff_date
    strike = row.STRIKE
    r = interest_rate.loc[final_date, "DGS1MO"]
    if r is None:
        r = prev_ir
    else:
        prev_ir = r
    # Filter df
    wk_df = spy.loc[:final_date,]
    # Generate simulation
    mmar = MMAR(wk_df["Close"], volume=wk_df["Volume"])
    simulation = mmar.get_MMAR_MC(S0, n=num_steps, num_sim=num_sim)[:, -1:]
    exp_price.append(simulation.mean())
    exp_price_std.append(simulation.std())
    exp_price_min.append(simulation.min())
    exp_price_max.append(simulation.max())
    q1, q2, q3 = np.percentile(simulation, q=[25, 50, 75])
    exp_price_median.append(q2)
    exp_price_q1.append(q1)
    exp_price_q3.append(q3)
    exp_price_kurtosis.append(st.kurtosis(simulation)[0])
    exp_price_skew.append(st.skew(simulation)[0])
    exp_call_price, prob = get_call_price(simulation, strike, r / 100, num_steps / 360)
    exp_call_prices.append(exp_call_price)
    strikes.append(strike)
    last_quotes.append(S0)
    days.append(num_steps)
    probs.append(prob)
    rates.append(r)
    date_idx.append(final_date)

  0%|          | 0/1508 [00:00<?, ?it/s]

The series has been adjusted: the orginal size was 3273, the new size is 3272 with 6 dividers.
The series has been adjusted: the orginal size was 3274, the new size is 3272 with 6 dividers.
The series has been adjusted: the orginal size was 3275, the new size is 3272 with 6 dividers.
The series has been adjusted: the orginal size was 3277, the new size is 3276 with 34 dividers.
The series has been adjusted: the orginal size was 3279, the new size is 3278 with 6 dividers.
The series has been adjusted: the orginal size was 3281, the new size is 3280 with 18 dividers.
The series has been adjusted: the orginal size was 3283, the new size is 3282 with 6 dividers.
The series has been adjusted: the orginal size was 3284, the new size is 3282 with 6 dividers.
The series has been adjusted: the orginal size was 3287, the new size is 3286 with 6 dividers.
The series has been adjusted: the orginal size was 3291, the new size is 3290 with 14 dividers.
The series has been adjusted: the orginal size 

In [31]:
exp_price_df = pd.DataFrame(
    {
        "last_quote": last_quotes,
        "strike": strikes,
        "exp_price": exp_price,
        "exp_call_price": exp_call_prices,
        "exp_price_min": exp_price_min,
        "exp_price_max": exp_price_max,
        "exp_price_std": exp_price_std,
        "exp_price_median": exp_price_median,
        "exp_price_q1": exp_price_q1,
        "exp_price_q3": exp_price_q3,
        "exp_price_kurtosis": exp_price_kurtosis,
        "exp_price_skew": exp_price_skew,
        "days": days,
        "r": rates,
        "prob_itm": probs,
    },
    index=pd.Index(name="Date", data=date_idx),
)

In [32]:
exp_price_df

,last_quote,strike,exp_price,exp_call_price,exp_price_min,exp_price_max,exp_price_std,exp_price_median,exp_price_q1,exp_price_q3,exp_price_kurtosis,exp_price_skew,days,r,prob_itm
Date,,,,,,,,,,,,,,,
2018-01-02,268.81,269.0,269.070010,4.789588,214.689394,325.831867,11.934435,268.799682,260.873449,276.974963,0.026867,0.135577,22,1.28,0.493064
2018-01-03,270.47,270.0,270.743106,4.909966,215.594853,336.233083,11.382304,270.522864,262.929824,278.284771,0.035900,0.128826,21,1.29,0.518110
2018-01-04,271.56,272.0,271.786576,4.329652,224.878945,327.091413,11.118207,271.549004,264.164504,279.174647,0.010345,0.121506,20,1.29,0.483688
2018-01-05,273.41,273.0,273.656825,4.908688,225.148320,335.682395,11.494269,273.430561,265.792843,281.251103,0.039568,0.128922,21,1.28,0.514954
2018-01-08,273.94,274.0,274.210037,4.865926,223.326384,340.838091,11.950461,273.952344,265.994513,282.126790,0.032291,0.132526,23,1.27,0.498324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,473.68,474.0,474.282017,10.017008,375.336770,605.674965,24.849107,473.631769,457.203817,490.677546,0.033999,0.158028,21,5.54,0.494068
2023-12-26,475.61,476.0,476.325901,10.575885,370.698376,613.852681,26.216615,475.609552,458.262271,493.650852,0.054380,0.166656,24,5.54,0.494202
2023-12-27,476.47,476.0,477.176954,10.893257,373.373723,613.593261,25.952014,476.473552,459.282456,494.247564,0.050592,0.165624,23,5.53,0.507330


In [33]:
exp_price_df.isna().sum()

last_quote            0
strike                0
exp_price             0
exp_call_price        0
exp_price_min         0
exp_price_max         0
exp_price_std         0
exp_price_median      0
exp_price_q1          0
exp_price_q3          0
exp_price_kurtosis    0
exp_price_skew        0
days                  0
r                     0
prob_itm              0
dtype: int64

In [34]:
# Save file
exp_price_df.to_csv("../data/processed/exp_price.csv")